## 0.0 setup

1. follow the setup instructions in `1_0_hybridization_setup.ipynb`

2. create directories and copy database files from the shared drive (`/srv/data/autumn_school/hybridization/`)

In [1]:
!rm -rf ~/hybridization_data/databases_hybrid_from_shared_drive 
!mkdir -p ~/hybridization_data/databases_hybrid_from_shared_drive
!mkdir -p ~/hybridization_data/databases_hybrid
!cp -a /srv/data/autumn_school/hybridization/databases_hybrid_from_shared_drive/* ~/hybridization_data/databases_hybrid_from_shared_drive

## 0.1. imports
### 0.1.1. regular imports

In [2]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import brightway2 as bw
import ecospold2matrix as e2m
import pymrio
# data science
import pandas as pd

Using environment variable BRIGHTWAY2_DIR for data directory:
/home/weinold/bw_data


### 0.1.2. local imports

In [3]:
sys.path.append(os.path.join(Path.home(), 'pylcaio', 'src')) # required for local import of pylcaio
import pylcaio

## 0.2. file paths
### 0.2.1. directories

In [4]:
%%capture
print(path_dir_databases_raw := os.path.join(Path.home(), 'hybridization_data/databases_raw'))
print(path_dir_databases_hybrid := os.path.join(Path.home(), 'hybridization_data/databases_hybrid'))
print(path_dir_databases_hybrid_from_shared_drive := os.path.join(Path.home(), 'hybridization_data/databases_hybrid_from_shared_drive'))

### 0.2.2. databases

In [5]:
%%capture
print(path_dir_ecoinvent_input := os.path.join(path_dir_databases_raw, 'ecoinvent-3.5-cutoff'))
print(path_file_hybrid_from_shared_drive := os.path.join(path_dir_databases_hybrid_from_shared_drive, 'pylcaio_class_instance_after_hybrid_ecoinvent_3_8.pkl'))
print(path_file_hybrid_analysis_from_shared_drive := os.path.join(path_dir_databases_hybrid_from_shared_drive, 'pylcaio_class_instance_after_analysis_ecoinvent_3_8.pkl'))

## 1.1. analysis

In [ ]:
with open(path_file_hybrid_analysis_from_shared_drive, 'rb') as filestream:
    analysis_object: pymrio.core.mriosystem.IOSystem = pd.read_pickle(filestream)

In [7]:
# this takes significant resources, you should instead load the relevant pickle file
#%%time
#analysis_object = pylcaio.Analysis(path_file_hybrid_from_shared_drive)
#analysis_object.calc_lifecycle()

In [ ]:
analysis_object.navigate_through_PRO_f(
    product = 'electricity',
    geography = 'FR',
    activity = 'hydro'
)

In [ ]:
process_only = analysis_object.C_f.dot(analysis_object.F_f.dot(analysis_object.Lp))
process_only = pd.DataFrame(process_only.todense(),
index=analysis_object.C_index, columns=analysis_object.PRO_f['activityName']).loc[:, analysis_object.hybridized_processes]

### 1.1.1. compare process-only and hybrid results

In [ ]:
uuid = 'b6095722-8eae-4102-b014-4386412d42e1_66c93e71-f32b-4591-901c-55395db5c132'
(analysis.D.loc[:,uuid]-process_only.loc[:,uuid])/process_only.loc[:,uuid]*100

## 2.1. export to `brightway` (currently broken)

compare GitHub issue in the `bw_hybrid` repo: [`export_to_brightway2` throws error `InvalidExchange`](https://github.com/michaelweinold/bw_hybrid/issues/26)

In [8]:
%%time
analysis_object.export_to_brightway2(
    bw2_project_name = 'banana',
    created_database_name = 'apple',
    path_to_ecoinvent_ecospold_datasets = path_dir_ecoinvent_input,
    aggregated = True
)

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds
Title: Writing activities to SQLite3 database:
  Started: 10/22/2022 15:27:18
  Finished: 10/22/2022 15:27:18
  Total time elapsed: 00:00:00
  CPU %: 100.40
  Memory %: 0.57
Created database: biosphere3
Creating default LCIA methods

Wrote 762 LCIA methods with 227223 characterization factors
Creating core data migrations

Extracting XML data from 0 datasets
Extracted 0 datasets in 0.17 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_co

WrongDatabase: Activity database must be hybrid-ecoinvent, but set() was also found